In [ ]:
%autoreload 2

In [ ]:
import logging 
logging.basicConfig()
# log = logging.getLogger("blocking").setLevel("DEBUG")
# log = logging.getLogger("sql").setLevel("DEBUG")
# log = logging.getLogger("gammas").setLevel("DEBUG")
# log = logging.getLogger("expectation_step").setLevel("DEBUG")
# log = logging.getLogger("maximisation_step").setLevel("DEBUG")
# log = logging.getLogger("comparison_evaluation").setLevel("DEBUG")

In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f

# WARNING:
# These config options are appropriate only if you're running Spark locally!!!
conf=SparkConf()
conf.set('spark.driver.memory', '8g')
conf.set("spark.sql.shuffle.partitions", "8") 

sc = SparkContext.getOrCreate(conf=conf)

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
import sys 
sys.path.append("sparklink")
from sql import *
from blocking import *
from gammas import *
from params import *
from expectation_step import *
from maximisation_step import *
from iterate import *
from comparison_evaluation import *


In [ ]:
df = spark.read.parquet("data/fake_1000.parquet")
df = df.dropDuplicates()

In [ ]:
blocking_rules = []

blocking_rules.append('l.first_name = r.first_name  and  l.dob = r.dob')
blocking_rules.append('l.first_name = r.first_name  and  l.city = r.city')
blocking_rules.append('l.first_name = r.first_name  and  l.email = r.email')

blocking_rules.append('l.surname = r.surname  and  l.dob = r.dob')
blocking_rules.append('l.surname = r.surname  and  l.city = r.city')
blocking_rules.append('l.surname = r.surname  and  l.email = r.email')

blocking_rules.append('l.dob = r.dob  and  l.city = r.city')
blocking_rules.append('l.dob = r.dob  and  l.email = r.email')

blocking_rules.append('l.city = r.city  and  l.email = r.email')

In [ ]:
# df_comparison = cartestian_block(df, spark=spark)
df_comparison = block_using_rules(df, blocking_rules, spark=spark)
df_comparison = df_comparison.withColumn("label", (df_comparison["group_l"]==df_comparison["group_r"]).cast("int"))

In [ ]:
import copy
from gammas import *

gamma_settings = {
    "first_name": {
        "levels": 3,
    },
    "surname": {
        "levels": 3
    },
    "dob": {
        "levels": 3
    },
    "city": {
        "levels": 3
    },
    "email": {
        "levels": 3
    }
}

df_gammas = add_gammas(df_comparison, gamma_settings, spark, include_orig_cols = False)
params = Params(gamma_settings, starting_lambda=0.5)

real_params = get_real_params(df_comparison, df_gammas, spark, params)
params.real_params = real_params
# params.params = copy.deepcopy(real_params)

df_gammas = add_gammas(df_comparison, gamma_settings, spark, include_orig_cols = True)
df_gammas.show()
df_gammas.persist()

In [ ]:
import copy 

params = Params(gamma_settings, starting_lambda=0.8)

params.set_pi_value("gamma_0", 0, "match", 0.2)
params.set_pi_value("gamma_0", 1, "match", 0.8)

params.set_pi_value("gamma_0", 0, "non_match", 0.8)
params.set_pi_value("gamma_0", 1, "non_match", 0.2)

params.set_pi_value("gamma_1", 0, "match", 0.2)
params.set_pi_value("gamma_1", 1, "match", 0.4)
params.set_pi_value("gamma_1", 2, "match", 0.4)

params.set_pi_value("gamma_1", 0, "non_match", 0.8)
params.set_pi_value("gamma_1", 1, "non_match", 0.15)
params.set_pi_value("gamma_1", 2, "non_match", 0.05)

# real_params = get_real_params(df_comparison, df_gammas, spark, params)
# params.real_params = real_params
# params.params = copy.deepcopy(real_params)
params.probability_distribution_chart()





In [ ]:
df_e = iterate(df_gammas, spark, params, num_iterations=5, print_ll=True)

In [ ]:
params.lambda_iteration_chart()

In [ ]:
params.pi_iteration_chart()

In [ ]:
sql = sql_gen_gamma_prob_columns(df_gammas, params)
df_gammas.registerTempTable("df_with_gamma")
df_with_gamma_probs = spark.sql(sql)
calculate_log_likelihood_df(df_with_gamma_probs, params, spark).show(30)
get_overall_log_likelihood(df_with_gamma_probs, params, spark)

In [ ]:
# Compute real params
real_params = get_real_params(df_comparison, df_gammas, spark, params)
params.real_params = real_params
params.params = copy.deepcopy(real_params)

sql = sql_gen_gamma_prob_columns(df_gammas, params)
df_gammas.registerTempTable("df_with_gamma")
df_with_gamma_probs = spark.sql(sql)
calculate_log_likelihood_df(df_with_gamma_probs, params, spark).show(30)
get_overall_log_likelihood(df_with_gamma_probs, params, spark)

# df_e = run_expectation_step(df_gammas, spark, params, print_ll=True)

In [ ]:
params.lambda_iteration_chart()

In [ ]:
params.pi_iteration_chart()

In [ ]:
params.probability_distribution_chart()

In [ ]:
## Get selection of false positive true negative etc

In [ ]:
1+1